In [ ]:
import os
import gensim
import nltk
from nltk import word_tokenize
import re

In [ ]:
#读取所有文件名并进行分词
folder_path = 'E:/Desktop/papers/all_papers/'
word_list = []
for fname in os.listdir(folder_path):
    cutwords = word_tokenize(fname)
#     print(cutwords)
    for i in cutwords:
        t = re.sub('.txt','',i) #去除.txt后缀
        word_list.append(t.lower())
    with open(folder_path + fname,encoding='utf-8',errors='ignore') as f:
        abtract = f.read()
    cut_abtract_words = word_tokenize(abtract)
    for i in cut_abtract_words:
        word_list.append(i.lower())       
word_list

In [ ]:
#去除标点符号
interpunctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']   #定义标点符号列表
cutwords1 = [word for word in word_list if word not in interpunctuations]
len(cutwords1),cutwords1

In [ ]:
#去停用词
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
cutwords2 = [word for word in cutwords1 if word not in stops]
len(cutwords2),cutwords2

In [ ]:
#词干提取
from nltk.stem import PorterStemmer
cutwords3 = []
for cutword in cutwords2:
    cutwords3.append(PorterStemmer().stem(cutword))
len(cutwords3)

In [ ]:
#词性还原
from nltk import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
cutwords4 = []
for cutword in cutwords3:
    cutwords4.append(WordNetLemmatizer().lemmatize(cutword, pos='n'))
len(cutwords4)

In [ ]:
#读进文件
with open('E:/Desktop/papers/title_word_dict.txt', 'wb') as fw:
    for i in range(len(cutwords4)):
        fw.write(cutwords1[i].encode('utf-8'))
        fw.write('\n'.encode('utf-8'))
fw.close()

In [ ]:
#训练词向量
from gensim.models.word2vec import LineSentence
import gensim.models as word2vec

sentences = LineSentence('E:/Desktop/papers/title_word_dict.txt')
model = word2vec.Word2Vec(sentences, vector_size=100, sg=1, window=5,min_count=3,workers=4)
model.save('E:/Desktop/papers/title_word2vec.model')
model.wv.save_word2vec_format('E:/Desktop/papers/title_corpusDone.vector', binary=False)

In [ ]:
#加载模型
w2v_path = 'E:/Desktop/papers/title_word2vec.model'
def load_word2vec_model(w2_path):
    model = word2vec.Word2Vec.load(w2_path)
    return model
model = load_word2vec_model(w2v_path)
print(model.wv['pre-train'])

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def plot_with_labels(low_dim_embs, labels, filename):   # 绘制词向量图
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    print('绘制词向量中......')
    plt.figure(figsize=(10, 10))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)	# 画点，对应low_dim_embs中每个词向量
        plt.annotate(label,	# 显示每个点对应哪个单词
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.savefig(filename)
    plt.show()

In [ ]:
import numpy as np
def load_txt_vec(file, threshold=0, dtype='float'):	# 读取txt文件格式的word2vec词向量
    print('读取词向量文件中......')
    header = file.readline().split(' ')
    count = int(header[0]) if threshold <= 0 else min(threshold, int(header[0]))
    dim = int(header[1])
    words = []
    matrix = np.empty((count, dim), dtype=dtype)
    for i in range(count):
        word, vec = file.readline().split(' ', 1)
        words.append(word)
        matrix[i] = np.fromstring(vec, sep=' ', dtype=dtype)
    return (words, matrix)

In [ ]:
if __name__ == '__main__':
    w2v_txt_file = open('E:/Desktop/papers/title_corpusDone.vector','r',encoding='utf-8')
    words, vectors = load_txt_vec(w2v_txt_file)
    tsne = TSNE(perplexity=50,n_iter=5000,method='exact',n_components=2)
    plot_only = 200
    low_dim_embs = tsne.fit_transform(vectors[:plot_only])
    labels = [words[i] for i in range(plot_only)]
    plot_with_labels(low_dim_embs, labels, 'w2v.png')

In [ ]:
#统计词频
d = {}
for i in cutwords4:
    if i in d:
        d[i] += 1
    else:
        d[i] = 1
d

In [ ]:
#将字典转换成列表，并排序
items = list(d.items())
items.sort(key = lambda x:x[1], reverse=True)

items

In [ ]:
chiyun = []
for i in range(100):
    word, count = items[i]
    chiyun.append(word)
len(chiyun)

In [ ]:
#词云展示
from wordcloud import WordCloud

text_cut = '/'.join(chiyun)
wcloud = WordCloud(background_color='white', width=1000,max_words=1000, height=800, margin=2).generate(text_cut)
plt.imshow(wcloud)
plt.axis('off')
plt.show()